In [61]:
from dotenv import load_dotenv

load_dotenv()

True

In [62]:
from langsmith import Client

example_inputs = [
  ("2024년 제22회 SW역량검증(TOPCIT) 정기평가의 평가일시는 언제인가요?",
   "2024년 10월 12일 토요일 오전 9시 30분부터 12시까지입니다. 고사장 입실 완료 시간은 9시 10분입니다."),
  ("SW역량검증(TOPCIT) 정기평가에 단체 접수한 학생이 반드시 해야 할 절차는 무엇인가요?",
   "(가) 구글 폼 신청과 (나) TOPCIT 홈페이지에서 신청자 정보등록을 모두 완료해야 시험에 응시할 수 있습니다."),
  ("SW역량검증(TOPCIT) 정기평가 단체접수의 등록기간은 언제인가요?",
   "등록기간은 2024년 9월 12일(목)부터 9월 19일(목) 오후 3시까지입니다."),
  ("SW역량검증(TOPCIT) 정기평가 단체접수 대상자가 응시를 완료하면 어떤 혜택을 받을 수 있나요?",
   "응시료 전액 지원과 함께 SW 마일리지 20점이 지급됩니다."),
  ("SW역량검증(TOPCIT) 정기평가에 응시하지 않은 학생은 어떻게 되나요?",
   "신청 후 불응시한 학생은 SW 마일리지에서 10점이 차감됩니다."),
  ("SW역량검증(TOPCIT) 정기평가 단체접수 신청자는 어디서 시험을 보게 되나요?",
   "단체접수용 시험장은 경북대학교 내 실습장으로 배정되며, 정확한 장소는 수험표 발급기간에 확인할 수 있습니다."),
  ("SW역량검증(TOPCIT) 정기평가 단체접수 신청은 어떻게 진행되나요?",
   "구글 폼 링크를 통해 신청한 후, TOPCIT 홈페이지에서 신청자 정보를 등록해야 합니다."),
  ("SW역량검증(TOPCIT) 정기평가 관련 문의는 어디로 해야 하나요?",
   "소프트웨어교육원(전화: 053-950-7670, 이메일: bk6722@knu.ac.kr)으로 문의할 수 있습니다."),
]

client = Client()
dataset_name = "TOPCIT"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Questions and answers about TOPCIT",
)
for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        metadata={"source": "공지사항"},
        dataset_id=dataset.id,
    )

In [63]:
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

In [64]:
### INDEX

load_dotenv()
# Load docs
loader = WebBaseLoader(
    web_paths=("https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=28223",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"id": ["bo_v_con", "bo_v_title"]},
        )
    )
)
docs = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

splits = text_splitter.split_documents(docs)

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

# Embed and store in Chroma
index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

database = PineconeVectorStore.from_documents(splits, embeddings, index_name=index_name)

# Index
retriever=database.as_retriever()

In [65]:
from langchain_upstage import ChatUpstage
from langchain import hub

class RagBot:

    def __init__(self, retriever, model: str = "upstage-model"):
        self._retriever = retriever
        self._client = ChatUpstage(model=model)
        self._model = model

    @traceable()
    def retrieve_docs(self, question):
        # 문서 검색 수행
        return self._retriever.invoke(question)

    @traceable()
    def invoke_llm(self, question, docs):
        # Upstage LLM 호출
        response = self._client.generate(  # 'chat' -> 'generate'
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "당신은 공지사항의 내용을 정확히 알려주는 직원입니다. "
                               "다음 문서들을 사용하여 사용자 질문에 대한 간결한 답변을 작성하십시오.\n\n"
                               f"## Docs\n\n{docs}",
                },
                {"role": "user", "content": question},
            ],
        )

        # 응답 구조 확인 후 수정
        # 예를 들어, Upstage의 응답이 choices 리스트를 포함하고 있을 수 있습니다.
        answer = response['choices'][0]['message']['content']
        return {
            "answer": answer,
            "contexts": [str(doc) for doc in docs],
        }

    @traceable()
    def get_answer(self, question: str):
        docs = self.retrieve_docs(question)
        return self.invoke_llm(question, docs)

rag_bot = RagBot(retriever)


In [66]:
def predict_rag_answer(example: dict):
    """답변만 평가할 때 사용"""
    response = rag_bot.get_answer(example["question"])  # 'input_question' -> 'question'
    return {"answer": response["answer"]}

def predict_rag_answer_with_context(example: dict):
    """Context를 활용해서 hallucination을 평가할 때 사용"""
    response = rag_bot.get_answer(example["question"])  # 'input_question' -> 'question'
    return {"answer": response["answer"], "contexts": response["contexts"]}


In [67]:
from langchain_upstage import ChatUpstage
from langchain import hub

# Grade prompt
grade_prompt_answer_accuracy = hub.pull("langchain-ai/rag-answer-vs-reference")  # Upstage에서 제공하는 prompt

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("question")
    reference = example.outputs.get("answer")
    prediction = run.outputs.get("answer")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")  # Upstage LLM 모델 설정

    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}


In [68]:
from langchain_upstage import ChatUpstage
from langchain import hub

# Prompt
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")  # Upstage에서 제공하는 프롬프트

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """

    # RAG inputs
    input_question = example.inputs.get("question")
    contexts = run.outputs.get("contexts")

    # RAG answer
    prediction = run.outputs.get("answer")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")  # Upstage LLM 모델 설정

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}


In [69]:
from langchain_upstage import ChatUpstage
from langchain import hub

# Grade prompt
grade_prompt_answer_helpfulness = hub.pull("langchain-ai/rag-answer-helpfulness")  # Upstage에서 제공하는 프롬프트

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question and RAG chain answer
    input_question = example.inputs.get("question")
    prediction = run.outputs.get("answer")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")  # Upstage LLM 모델 설정

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}


In [70]:
from langsmith.evaluation import evaluate

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_helpfulness_evaluator],
    experiment_prefix="topcit_evaluate",
    metadata={"version": "topcit, upstage-model"},
)

View the evaluation results for experiment: 'topcit_evaluate-bf55e5b9' at:
https://smith.langchain.com/o/410ab3fd-ee98-5d8d-81e6-22756766a30e/datasets/4088c41c-9f9d-48fc-830d-29620d55df41/compare?selectedSessions=fbbcb854-a053-4222-ba73-9eac039a75b4




0it [00:00, ?it/s]

Error running target function: 'str' object has no attribute 'content'
Error running evaluator <DynamicRunEvaluator answer_evaluator> on run 04aca7aa-f920-4eb8-a1d4-3ba4733a6a1c: BadRequestError('Bad Request')
Traceback (most recent call last):
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\_runner.py", line 1325, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 643, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [72]:
from dotenv import load_dotenv
from langsmith import Client
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
from langchain_upstage import ChatUpstage
from langchain import hub

# Load environment variables
load_dotenv()

# Create dataset in LangSmith
example_inputs = [
    ("2024년 제22회 SW역량검증(TOPCIT) 정기평가의 평가일시는 언제인가요?",
     "2024년 10월 12일 토요일 오전 9시 30분부터 12시까지입니다. 고사장 입실 완료 시간은 9시 10분입니다."),
    ("SW역량검증(TOPCIT) 정기평가에 단체 접수한 학생이 반드시 해야 할 절차는 무엇인가요?",
     "(가) 구글 폼 신청과 (나) TOPCIT 홈페이지에서 신청자 정보등록을 모두 완료해야 시험에 응시할 수 있습니다."),
    ("SW역량검증(TOPCIT) 정기평가 단체접수의 등록기간은 언제인가요?",
     "등록기간은 2024년 9월 12일(목)부터 9월 19일(목) 오후 3시까지입니다."),
    ("SW역량검증(TOPCIT) 정기평가 단체접수 대상자가 응시를 완료하면 어떤 혜택을 받을 수 있나요?",
     "응시료 전액 지원과 함께 SW 마일리지 20점이 지급됩니다."),
    ("SW역량검증(TOPCIT) 정기평가에 응시하지 않은 학생은 어떻게 되나요?",
     "신청 후 불응시한 학생은 SW 마일리지에서 10점이 차감됩니다."),
    ("SW역량검증(TOPCIT) 정기평가 단체접수 신청자는 어디서 시험을 보게 되나요?",
     "단체접수용 시험장은 경북대학교 내 실습장으로 배정되며, 정확한 장소는 수험표 발급기간에 확인할 수 있습니다."),
    ("SW역량검증(TOPCIT) 정기평가 단체접수 신청은 어떻게 진행되나요?",
     "구글 폼 링크를 통해 신청한 후, TOPCIT 홈페이지에서 신청자 정보를 등록해야 합니다."),
    ("SW역량검증(TOPCIT) 정기평가 관련 문의는 어디로 해야 하나요?",
     "소프트웨어교육원(전화: 053-950-7670, 이메일: bk6722@knu.ac.kr)으로 문의할 수 있습니다."),
]

client = Client()
dataset_name = "TOPCIT_1"

dataset = client.create_dataset(
    dataset_name=dataset_name, description="Questions and answers about TOPCIT",
)
for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        metadata={"source": "공지사항"},
        dataset_id=dataset.id,
    )

# Load docs
loader = WebBaseLoader(
    web_paths=("https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=28223",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"id": ["bo_v_con", "bo_v_title"]},
        )
    )
)
docs = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

splits = text_splitter.split_documents(docs)

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

# Embed and store in Pinecone
index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

database = PineconeVectorStore.from_documents(splits, embeddings, index_name=index_name)

retriever = database.as_retriever()

class RagBot:

    def __init__(self, retriever, model: str = "upstage-model"):
        self._retriever = retriever
        self._client = ChatUpstage(model=model)
        self._model = model

    def retrieve_docs(self, question):
        # Document search
        return self._retriever.invoke(question)

    def invoke_llm(self, question, docs):
        # Call Upstage LLM
        response = self._client.generate(
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "당신은 공지사항의 내용을 정확히 알려주는 직원입니다. "
                               "다음 문서들을 사용하여 사용자 질문에 대한 간결한 답변을 작성하십시오.\n\n"
                               f"## Docs\n\n{docs}",
                },
                {"role": "user", "content": question},
            ],
        )

        # Debug response
        print("Response:", response)

        # Extract answer from response
        answer = response['choices'][0]['message']['content']
        return {
            "answer": answer,
            "contexts": [str(doc) for doc in docs],
        }

    def get_answer(self, question: str):
        docs = self.retrieve_docs(question)
        return self.invoke_llm(question, docs)

rag_bot = RagBot(retriever)

def predict_rag_answer(example: dict):
    """답변만 평가할 때 사용"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"]}

def predict_rag_answer_with_context(example: dict):
    """Context를 활용해서 hallucination을 평가할 때 사용"""
    response = rag_bot.get_answer(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

# Grade prompt
grade_prompt_answer_accuracy = hub.pull("langchain-ai/rag-answer-vs-reference")  # Upstage에서 제공하는 prompt

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("question")
    reference = example.outputs.get("answer")
    prediction = run.outputs.get("answer")

    # Debug information
    print(f"Evaluating answer for question: {input_question}")
    print(f"Reference answer: {reference}")
    print(f"Prediction: {prediction}")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")

    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}

# Prompt
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")  # Upstage에서 제공하는 프롬프트

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """

    # RAG inputs
    input_question = example.inputs.get("question")
    contexts = run.outputs.get("contexts")

    # RAG answer
    prediction = run.outputs.get("answer")

    # Debug information
    print(f"Evaluating hallucinations for question: {input_question}")
    print(f"Contexts: {contexts}")
    print(f"Prediction: {prediction}")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

# Grade prompt
grade_prompt_answer_helpfulness = hub.pull("langchain-ai/rag-answer-helpfulness")  # Upstage에서 제공하는 프롬프트

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question and RAG chain answer
    input_question = example.inputs.get("question")
    prediction = run.outputs.get("answer")

    # Debug information
    print(f"Evaluating helpfulness for question: {input_question}")
    print(f"Prediction: {prediction}")

    # Upstage grader
    llm = ChatUpstage(model="upstage-model")

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

from langsmith.evaluation import evaluate

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_helpfulness_evaluator],
    experiment_prefix="topcit_evaluate",
    metadata={"version": "topcit, upstage-model"},
)


View the evaluation results for experiment: 'topcit_evaluate-e8ef2af6' at:
https://smith.langchain.com/o/410ab3fd-ee98-5d8d-81e6-22756766a30e/datasets/b04f3efe-fca1-4de9-9435-08b65c42b908/compare?selectedSessions=e270c46a-7fca-47ae-8321-2e7fa4d7f8a1




0it [00:00, ?it/s]

Error running target function: 'str' object has no attribute 'content'


Evaluating answer for question: SW역량검증(TOPCIT) 정기평가 단체접수의 등록기간은 언제인가요?
Reference answer: 등록기간은 2024년 9월 12일(목)부터 9월 19일(목) 오후 3시까지입니다.
Prediction: None


Error running evaluator <DynamicRunEvaluator answer_evaluator> on run b04d9b2e-c2e0-4817-b48d-382a0a493a1c: BadRequestError('Bad Request')
Traceback (most recent call last):
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\_runner.py", line 1325, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 643, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEESEOKHYUN\AppData\Local\Temp\ipykernel_9600\3503254

...

Error running evaluator <DynamicRunEvaluator answer_evaluator> on run 6471e184-6ccf-47d2-90ac-132ce0a5daa0: BadRequestError('Bad Request')
Traceback (most recent call last):
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\_runner.py", line 1325, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 643, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEESEOKHYUN\AppData\Local\Temp\ipykernel_9600\3503254

Evaluating helpfulness for question: 2024년 제22회 SW역량검증(TOPCIT) 정기평가의 평가일시는 언제인가요?
Prediction: None
Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가에 단체 접수한 학생이 반드시 해야 할 절차는 무엇인가요?
Prediction: None
Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가 단체접수 신청은 어떻게 진행되나요?
Prediction: None
Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가 단체접수 신청자는 어디서 시험을 보게 되나요?
Prediction: None
Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가 관련 문의는 어디로 해야 하나요?
Prediction: None


Error running evaluator <DynamicRunEvaluator answer_evaluator> on run ad62d480-fd15-49e3-a840-5c3c4a39b5ee: BadRequestError('Bad Request')
Traceback (most recent call last):
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\_runner.py", line 1325, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 643, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEESEOKHYUN\AppData\Local\Temp\ipykernel_9600\3503254

Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가 단체접수 대상자가 응시를 완료하면 어떤 혜택을 받을 수 있나요?
Prediction: None
Evaluating helpfulness for question: SW역량검증(TOPCIT) 정기평가에 응시하지 않은 학생은 어떻게 되나요?
Prediction: None


Error running evaluator <DynamicRunEvaluator answer_helpfulness_evaluator> on run 6471e184-6ccf-47d2-90ac-132ce0a5daa0: BadRequestError('Bad Request')
Traceback (most recent call last):
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\_runner.py", line 1325, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\LEESEOKHYUN\anaconda3\Lib\site-packages\langsmith\run_helpers.py", line 643, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEESEOKHYUN\AppData\Local\Temp\ipykernel_